In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("treatment%_per_state_2016_to_2022.csv")
df

,Year,FIPSST,Medi_or_Behav_Treat%,Medi_Treat%,Behav_Treat%,Medi&Behav_Treat%
0,2016,1,85.148515,77.227723,37.623762,29.702970
1,2016,2,71.428571,53.571429,50.000000,32.142857
2,2016,4,76.250000,57.500000,53.750000,35.000000
3,2016,5,74.509804,60.784314,47.058824,33.333333
4,2016,6,60.000000,35.000000,51.666667,26.666667
...,...,...,...,...,...,...
352,2022,51,71.264368,59.770115,41.379310,29.885057
353,2022,53,62.666667,45.333333,40.000000,22.666667
354,2022,54,68.595041,56.198347,38.016529,25.619835
355,2022,55,73.913043,60.869565,52.173913,39.130435


In [3]:


# Map state FIPS code to state abbreviations
state_abbr = {
    1: 'AL', 2: 'AK', 4: 'AZ', 5: 'AR', 6: 'CA', 8: 'CO', 9: 'CT', 10: 'DE', 11: 'DC',
    12: 'FL', 13: 'GA', 15: 'HI', 16: 'ID', 17: 'IL', 18: 'IN', 19: 'IA', 20: 'KS',
    21: 'KY', 22: 'LA', 23: 'ME', 24: 'MD', 25: 'MA', 26: 'MI', 27: 'MN', 28: 'MS',
    29: 'MO', 30: 'MT', 31: 'NE', 32: 'NV', 33: 'NH', 34: 'NJ', 35: 'NM', 36: 'NY',
    37: 'NC', 38: 'ND', 39: 'OH', 40: 'OK', 41: 'OR', 42: 'PA', 44: 'RI', 45: 'SC',
    46: 'SD', 47: 'TN', 48: 'TX', 49: 'UT', 50: 'VT', 51: 'VA', 53: 'WA', 54: 'WV',
    55: 'WI', 56: 'WY'
}

# Map state FIPSST to state abbreviation
df['state'] = df['FIPSST'].map(state_abbr)

df


,Year,FIPSST,Medi_or_Behav_Treat%,Medi_Treat%,Behav_Treat%,Medi&Behav_Treat%,state
0,2016,1,85.148515,77.227723,37.623762,29.702970,AL
1,2016,2,71.428571,53.571429,50.000000,32.142857,AK
2,2016,4,76.250000,57.500000,53.750000,35.000000,AZ
3,2016,5,74.509804,60.784314,47.058824,33.333333,AR
4,2016,6,60.000000,35.000000,51.666667,26.666667,CA
...,...,...,...,...,...,...,...
352,2022,51,71.264368,59.770115,41.379310,29.885057,VA
353,2022,53,62.666667,45.333333,40.000000,22.666667,WA
354,2022,54,68.595041,56.198347,38.016529,25.619835,WV
355,2022,55,73.913043,60.869565,52.173913,39.130435,WI


In [4]:
df.rename(columns={'FIPSST': 'id'}, inplace=True)

In [5]:
import altair as alt
from vega_datasets import data

states = alt.topo_feature(data.us_10m.url, 'states')
source = df


column_name_select = alt.binding_select(
    options=['Medi_or_Behav_Treat%', 'Medi_Treat%', 'Behav_Treat%', 'Medi&Behav_Treat%'],
    name='Select a column: '
)

selected_column_param = alt.param(value="Medi_or_Behav_Treat%", bind=column_name_select)

year_slider = alt.binding_range(name="Select a year:", min=df["Year"].min(), max=df["Year"].max(), step=1)
selected_year = alt.selection_point(fields=["Year"], bind=year_slider)



# Create a selection that chooses the clicked state
click = alt.selection_point(fields=['id'], empty=False)

# Define the color scale for the map
color_scale = alt.Scale(domain=[20, 90], scheme='blueorange')

# Create the choropleth map
states = alt.topo_feature(data.us_10m.url, 'states')

map_chart = alt.Chart(source).mark_geoshape().encode(
    shape='geo:G',
    color=alt.Color("selected_column:Q", scale=color_scale),
    tooltip=[
        alt.Tooltip("state:N", title="State"),
        alt.Tooltip("selected_column:Q", title="Treatment Prevalence (%)", format=".2f")
    ]
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(data=states, key='id'),
    as_='geo'
).transform_calculate(
    selected_column=f"datum[{selected_column_param.name}]"
).transform_filter(
    selected_year
).add_params(
    selected_year,
    selected_column_param,
    click
).properties(
    width=800,
    height=500,
    title="ADHD Treatment by State"
).project(
    type="albersUsa"
)




# Pivot the data for the bar chart
long_df = source.melt(id_vars=['Year', 'id', 'state'], 
                      value_vars=['Medi_or_Behav_Treat%', 'Medi_Treat%', 'Behav_Treat%', 'Medi&Behav_Treat%'],
                      var_name='Treatment', 
                      value_name='Percentage')

#set up barchart scale
min_percentage = 0   # Adjust this value as needed
max_percentage = 100 # Adjust this value as needed

# Create the bar chart
bar_chart = alt.Chart(long_df).mark_bar().encode(
    x='Treatment:N',
    #y='Percentage:Q',
    y=alt.Y('Percentage:Q', scale=alt.Scale(domain=[0, 100]), axis=alt.Axis(title='Percentage')),
 
    color='Treatment:N',
    column='state:N',
    tooltip=[
        alt.Tooltip('Year:O', title='Year'),
        alt.Tooltip('Percentage:Q', title='Percentage', format='.2f')
    ]
).transform_filter(
    selected_year
).transform_filter(
    click
).properties(
    width=150,
    height=400
)



year_trend_chart = alt.Chart(long_df).mark_bar().encode(
    x='Year:O',
    y=alt.Y('Percentage:Q', scale=alt.Scale(domain=[0, 100]), axis=alt.Axis(title='Percentage')),
    color='Year:O',
    column='state:N',
    tooltip=[
        alt.Tooltip('Year:O', title='Year'),
        alt.Tooltip('Percentage:Q', title='Percentage', format='.2f')
    ]
).transform_filter(
    (alt.datum.Treatment == "Medi_or_Behav_Treat%") 
).transform_filter(
    click
).properties(
    width=150,
    height=200,
    title="Yearly Trend of 'Medi_or_Behav_Treat%' Treatment"
)


# Combine the charts
combined_chart = alt.vconcat(map_chart, year_trend_chart,bar_chart).resolve_scale(
    color='independent',
    x='independent'
).add_params(
    selected_year,
    click
)

combined_chart


alt.VConcatChart(...)

In [6]:
import altair as alt
from vega_datasets import data

column_name_select = alt.binding_select(
    options=['Medi_or_Behav_Treat%', 'Medi_Treat%', 'Behav_Treat%', 'Medi&Behav_Treat%'],
    name='Select a column: '
)

selected_column_param = alt.param(value="Medi_or_Behav_Treat%", bind=column_name_select)

year_slider = alt.binding_range(name="Select a year:", min=df["Year"].min(), max=df["Year"].max(), step=1)
selected_year = alt.selection_point(fields=["Year"], bind=year_slider)

states = alt.topo_feature(data.us_10m.url, 'states')
source = df
 
alt.Chart(source).mark_geoshape().encode(
    shape='geo:G',
    color=alt.Color("selected_column:Q", title="Treatment Prevalence (%)",scale=alt.Scale(scheme="bluegreen")),
    tooltip=[
        alt.Tooltip("state:N", title="State"),
        alt.Tooltip("selected_column:Q", title="Treatment Prevalence (%)", format=".2f")
    ]
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(data=states, key='id'),
    as_='geo'
).transform_calculate(
    selected_column=f"datum[{selected_column_param.name}]"
).transform_filter(
    selected_year
).properties(
    width=800,
    height=500,
    title="ADHD Treatment by State"
).add_params(
    selected_year,
    selected_column_param
).project(
    type="albersUsa"
)


alt.Chart(...)

In [7]:
import altair as alt
from vega_datasets import data



column_name_select = alt.binding_select(
    options=['Medi_or_Behav_Treat%', 'Medi_Treat%', 'Behav_Treat%', 'Medi&Behav_Treat%'],
    name='Select a column: '
)

selected_column_param = alt.param(value="Medi_or_Behav_Treat%", bind=column_name_select)

year_slider = alt.binding_range(name="Select a year:", min=df["Year"].min(), max=df["Year"].max(), step=1)
selected_year = alt.selection_point(fields=["Year"], bind=year_slider)


states = alt.topo_feature(data.us_10m.url, 'states')
source = df

 
alt.Chart(source).mark_geoshape().encode(
    shape='geo:G',
    color=alt.Color("selected_column:Q", title="Treatment Prevalence (%)",scale=alt.Scale(domain=[20, 90], scheme='blueorange')),
    tooltip=[
        alt.Tooltip("state:N", title="State"),
        alt.Tooltip("selected_column:Q", title="Treatment Prevalence (%)", format=".2f")
    ]
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(data=states, key='id'),
    as_='geo'
).transform_calculate(
    selected_column=f"datum[{selected_column_param.name}]"
).transform_filter(
    selected_year
).properties(
    width=800,
    height=500,
    title="ADHD Treatment by State"
).add_params(
    selected_year,
    selected_column_param
).project(
    type="albersUsa"
)



alt.Chart(...)

In [8]:
import altair as alt
from vega_datasets import data

# Assume 'df' is your DataFrame with 'state', 'Year', and other percentage columns

column_name_select = alt.binding_select(
    options=['Medi_or_Behav_Treat%', 'Medi_Treat%', 'Behav_Treat%', 'Medi&Behav_Treat%'],
    name='Select a column: '
)

selected_column_param = alt.param(value="Medi_or_Behav_Treat%", bind=column_name_select)

year_slider = alt.binding_range(name="Select a year:", min=df["Year"].min(), max=df["Year"].max(), step=1)
selected_year = alt.selection_point(fields=["Year"], bind=year_slider)



# Create a selection that chooses the clicked state
click = alt.selection_point(fields=['id'], empty=False)

# Define the color scale for the map
color_scale = alt.Scale(domain=[20, 90], scheme='blueorange')

# Create the choropleth map
states = alt.topo_feature(data.us_10m.url, 'states')

map_chart = alt.Chart(source).mark_geoshape().encode(
    shape='geo:G',
    color=alt.Color("selected_column:Q", scale=color_scale),
    tooltip=[
        alt.Tooltip("state:N", title="State"),
        alt.Tooltip("selected_column:Q", title="Treatment Prevalence (%)", format=".2f")
    ]
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(data=states, key='id'),
    as_='geo'
).transform_calculate(
    selected_column=f"datum[{selected_column_param.name}]"
).transform_filter(
    selected_year
).add_params(
    selected_year,
    selected_column_param,
    click
).properties(
    width=800,
    height=500,
    title="ADHD Treatment by State"
).project(
    type="albersUsa"
)




# Pivot the data for the bar chart
long_df = source.melt(id_vars=['Year', 'id', 'state'], 
                      value_vars=['Medi_or_Behav_Treat%', 'Medi_Treat%', 'Behav_Treat%', 'Medi&Behav_Treat%'],
                      var_name='Treatment', 
                      value_name='Percentage')

#set up barchart scale
min_percentage = 0   # Adjust this value as needed
max_percentage = 100 # Adjust this value as needed

# Create the bar chart
bar_chart = alt.Chart(long_df).mark_bar().encode(
    x='Treatment:N',
    #y='Percentage:Q',
    y=alt.Y('Percentage:Q', scale=alt.Scale(domain=[min_percentage, max_percentage]), axis=alt.Axis(title='Percentage')),
 
    color='Treatment:N',
    column='state:N'
).transform_filter(
    selected_year
).transform_filter(
    click
).properties(
    width=150,
    height=250
)


# Combine the charts
combined_chart = alt.vconcat(map_chart, bar_chart).resolve_scale(
    color='independent',
    x='independent'
)

combined_chart.add_params(
    selected_year,
    click
)

alt.VConcatChart(...)